In [1]:
import time
import MetaTrader5 as mt5

In [2]:
#initialise mt5 platform
if not mt5.initialize():
    print("Initialisation failed, error = ", mt5.last_error())
    
else:
    print("Initialisation Complete")

Initialisation Complete


In [ ]:
#Login details - Change when done with bot

login_id = 
password =
server = "
mt5.login(login_id,password,server)





True

In [4]:
# Account Info

account_info = mt5.account_info()
print("Name: ",account_info.name)
print("Balance: ",account_info.balance)
print("Equity:", account_info.equity)
print("Profit: ",account_info.profit)
print("Trade Mode: ",account_info.trade_mode)

Name:  Danial Nayyar
Balance:  10000.0
Equity: 10000.0
Profit:  0.0
Trade Mode:  0


In [ ]:
#mt5 initialised above
#login details adn account info done above as well

# MT5 Session Check + Signal Confirmation Terminal Info (WP1)

from datetime import datetime, timezone
import time
import pandas as pd
import zoneinfo

london = zoneinfo.ZoneInfo("Europe/London")

symbols = ["EURUSD","GBPUSD","USDCHF","GBPJPY","USDJPY","USDCAD","EURGBP", "GBPCAD", "XAUUSD"]


def session_check():
    terminal_info = mt5.terminal_info()
    trade_allowed = terminal_info.trade_allowed
    return trade_allowed

# M15 Close Detection 
def last_closed_M15_time(symbol):
    info = mt5.symbol_info(symbol)
    if info is None:
        if not mt5.symbol_select(symbol, True) or mt5.symbol_info(symbol) is None:
            raise RuntimeError(f"Symbol {symbol} not available")
    #most recent closed H1 bar
    rates = mt5.copy_rates_from_pos(symbol,mt5.TIMEFRAME_M15,0,1)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"No M15 bars for {symbol}")

    last_bar_time = rates[0]["time"]
    last_bar_datetime = datetime.utcfromtimestamp(last_bar_time)

    return last_bar_time, last_bar_datetime



def server_time_now(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        raise RuntimeError(f"No tick for {symbol}")
    return int(tick.time)



def rolling_mean(data, time_period):
    return pd.Series(data).rolling(time_period).mean().dropna().tolist()




def the_strategy(symbol):
    
    go_long, go_short = 0,0

    rates = mt5.copy_rates_from_pos(symbol,mt5.TIMEFRAME_M5, 0, 200)
    close_prices = [p["close"] for p in rates]
    
    #moving averages
    ma_8 = rolling_mean(close_prices, 8)
    ma_21 = rolling_mean(close_prices, 21)

    
    #cross_over check

    previous = ma_8[-2] - ma_21[-2]
    current = ma_8[-1] - ma_21[-1]

    cross_up = (previous <=0) and (current >0)
    cross_down = (previous >=0) and (current <0)

    if cross_up:
        go_long,go_short = 1,0
        print(f"{symbol} - Signal: LONG (SMA8 above SMA21)")
        
    elif cross_down:
        go_short, go_long = 1,0
        print(f"{symbol} - Signal: SHORT (SMA21 above SMA8)")

    print(symbol, "Diff b/w prev (t-1) & curr (t) MAs:", previous, current, "| long?", go_long, "short?", go_short)


    return go_long, go_short


def atr_sl_tp(symbol):
    period=14 
    lookback = 200
    
    rates = mt5.copy_rates_from_pos(symbol,mt5.TIMEFRAME_M15, 0, lookback)

    if rates is None or len(rates) ==0:
        raise RuntimeError(f"no bars for {symbol}")

    m = len(rates)
    if m< period+1:
        raise RuntimeError (f"Need a minimum of {period+1} bars for ATR({period}), got {m} instead")
    
    closes = [c["close"] for c in rates]
    highs = [h["high"] for h in rates]
    lows = [l["low"] for l in rates]

    tr = [0.0]*(m)
    for i in range(1,m):
        a = highs[i]-lows[i]
        b = abs(highs[i]- closes[i-1])
        c = abs(lows[i]-closes[i-1])
        tr[i] = max(a,b,c)


    atr = [float('nan')]*m
    seed = sum(tr[1:period+1])/period
    atr[period] = seed
    
    for i in range(period+1, m):
        atr[i] = (atr[i-1] * (period-1) +tr[i])/period

    last_atr_val = atr[-1]

    sl_distance = 1.5* last_atr_val
    tp_distance = 3.0* last_atr_val

    print(symbol, "ATR:", last_atr_val, "SLdist:", sl_distance, "TPdist:", tp_distance)


    return last_atr_val,sl_distance,tp_distance



    
def the_trade(symbol,go_long,go_short,sl_distance,tp_distance):
    lot = 1.0 
    point = mt5.symbol_info(symbol).point
    ask = mt5.symbol_info_tick(symbol).ask
    bid = mt5.symbol_info_tick(symbol).bid
    digits = mt5.symbol_info(symbol).digits
    deviation = 20

    request = None
    if go_long:
        enter = ask
        sl = round((ask - sl_distance)/point)*point
        tp =round((ask + tp_distance)/point)*point
        request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": enter,
    "sl": sl,
    "tp": tp,
    "deviation": deviation,
    "magic": 234001,
    "comment": "SMA STRAT- BUY",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,
    }
        
    elif go_short:
        enter = bid
        sl = round((bid + sl_distance)/point)*point
        tp = round((bid - tp_distance)/point)*point
        request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "price": enter,
    "sl": sl,
    "tp": tp,
    "deviation": deviation,
    "magic": 234002,
    "comment": "SMA STRAT- SELL",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,
}
    else: 
        return None
        
    result = mt5.order_send(request)

    direction = "LONG" if go_long else "SHORT"
    ok = (result.retcode == mt5.TRADE_RETCODE_DONE)
    status = "EXECUTED" if ok else "REJECTED" 
    
    return result



def heartbeat(ref_symbol):
    london = zoneinfo.ZoneInfo("Europe/London")
    last_timestamp_ref, last_bar_in_date_time = last_closed_M15_time(ref_symbol)
    if not session_check():
        print(f"No trade allowed- exiting loop")
        return
    while True:
        new_timestamp, new_datetime = last_closed_M15_time(ref_symbol)
        if new_timestamp != last_timestamp_ref:
            print(f"Da-dum, new M15 close detected @ {datetime.utcfromtimestamp(new_timestamp)}")
            # 10s wait for safety (for indicators and stuff)
            time.sleep(10)
            # strategy
            for symbol in symbols:
                go_long,go_short=the_strategy(symbol)
                if go_long or go_short:
                    try:
                        last_atr_val, sl_distance,tp_distance = atr_sl_tp(symbol)
                    except Exception as e:
                        print(f"{symbol} ATR Error: {e}")
                        continue
                    trade_result = the_trade(symbol,go_long,go_short,sl_distance,tp_distance)
                    if trade_result is not None:
                        print(f"{symbol} Info Post Order: Retcode = {trade_result.retcode}")
            #update time
            last_timestamp_ref = new_timestamp

        try:
            timestamp_now = server_time_now(ref_symbol)
        except RuntimeError as e:
            print(f"Error {e}")
            timestamp_now = int(time.time())

        next_target_run = last_timestamp_ref + 915 # 5m + 15s wait
        delay = max(15, next_target_run - timestamp_now) # 10s is the lower bound/minimum wait time, if (next_target_run - timestamp_now) < 5 
        print(f"Delay is: {delay} which is {delay/60} mins from")
        print(f"last_closed: {datetime.utcfromtimestamp(last_timestamp_ref)}")
        print(f"Next Wakeup: {datetime.utcfromtimestamp(next_target_run)}")
        
        #london_wake_up_time = datetime.utcfromtimestamp(next_target_run).replace(tzinfo = timezone.utc)
        #print(f"In London time the Wake up is: {london_wake_up_time.astimezone(london)}")
        
       #london_last_closed = datetime.utcfromtimestamp(last_timestamp_ref).replace(tzinfo = timezone.utc)
        #print(f"In London time the Last Close is: {london_last_closed.astimezone(london)}")
        
        time.sleep(delay)


#session_check() # Checked- It works
#filling_mode = symbol_sanity_check("EURUSD","filling_mode") # Checked - it works, its a tuple so index it like a list to get the actual value
#filling_mode = filling_mode[1]
#print(filling_mode)

#last_closed_H1_time("EURUSD") # Checked - it works

heartbeat(ref_symbol = "EURUSD")




Delay is: 620 which is 10.333333333333334 mins from
last_closed: 2025-11-20 13:30:00
Next Wakeup: 2025-11-20 13:45:15
Da-dum, new M15 close detected @ 2025-11-20 13:45:00
EURUSD Diff b/w prev (t-1) & curr (t) MAs: -0.00043148809523785303 -0.0005055952380950135 | long? 0 short? 0
GBPUSD Diff b/w prev (t-1) & curr (t) MAs: 0.0007002976190477739 0.0007362499999998828 | long? 0 short? 0
USDCHF Diff b/w prev (t-1) & curr (t) MAs: 0.0004394642857142861 0.00047250000000009784 | long? 0 short? 0
GBPJPY Diff b/w prev (t-1) & curr (t) MAs: -0.004821428571432307 -0.013148809523812588 | long? 0 short? 0
USDJPY Diff b/w prev (t-1) & curr (t) MAs: 0.07333333333332348 0.0692857142857406 | long? 0 short? 0
USDCAD Diff b/w prev (t-1) & curr (t) MAs: 9.16666666668231e-05 0.0001755952380952941 | long? 0 short? 0
EURGBP Diff b/w prev (t-1) & curr (t) MAs: 0.00010244047619034635 6.130952380956067e-05 | long? 0 short? 0
GBPCAD Diff b/w prev (t-1) & curr (t) MAs: -0.0003373214285713466 -0.0002779761904763145